In [1]:
!pip install groq python-dotenv numpy tqdm datasets

In [4]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

README.md: 0.00B [00:00, ?B/s]

main/train-00000-of-00001.parquet:   0%|          | 0.00/2.31M [00:00<?, ?B/s]

main/test-00000-of-00001.parquet:   0%|          | 0.00/419k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7473 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1319 [00:00<?, ? examples/s]

In [5]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [6]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm here to help you with any math problems you have. What kind of math are you working on? Do you have a specific problem you're stuck on or a concept you're trying to understand? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [7]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [8]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [9]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                predicted_answer = float(predicted_answer.replace(",", ""))
            
            diff = abs(predicted_answer - correct_answer)
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [10]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [11]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [18]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

  0%|          | 0/10 [00:00<?, ?it/s]

 50%|█████     | 5/10 [00:02<00:02,  1.76it/s]

Progress: [5/10]
Current Acc.: [40.00%]


100%|██████████| 10/10 [00:05<00:00,  1.90it/s]

Progress: [10/10]
Current Acc.: [30.00%]


In [ ]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

In [19]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_0.txt")

 10%|█         | 5/50 [00:01<00:12,  3.65it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:03<00:14,  2.75it/s]

Progress: [10/50]
Current Acc.: [40.00%]


 30%|███       | 15/50 [00:04<00:09,  3.62it/s]

...

Progress: [40/50]
Current Acc.: [22.50%]


 90%|█████████ | 45/50 [00:49<00:13,  2.75s/it]

Progress: [45/50]
Current Acc.: [24.44%]


100%|██████████| 50/50 [01:02<00:00,  1.26s/it]

Progress: [50/50]
Current Acc.: [26.00%]


In [20]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_3.txt")

 10%|█         | 5/50 [00:02<00:25,  1.79it/s]

Progress: [5/50]
Current Acc.: [20.00%]


 20%|██        | 10/50 [00:06<00:25,  1.56it/s]

Progress: [10/50]
Current Acc.: [30.00%]


 30%|███       | 15/50 [00:08<00:17,  2.01it/s]

...

Progress: [40/50]
Current Acc.: [17.50%]


 90%|█████████ | 45/50 [01:04<00:14,  2.83s/it]

Progress: [45/50]
Current Acc.: [17.78%]


100%|██████████| 50/50 [01:18<00:00,  1.57s/it]

Progress: [50/50]
Current Acc.: [20.00%]


In [21]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "direct_prompting_5.txt")

 10%|█         | 5/50 [00:05<00:51,  1.16s/it]

Progress: [5/50]
Current Acc.: [40.00%]


 20%|██        | 10/50 [00:08<00:30,  1.32it/s]

Progress: [10/50]
Current Acc.: [20.00%]


 30%|███       | 15/50 [00:11<00:22,  1.55it/s]

...

Progress: [40/50]
Current Acc.: [17.50%]


 90%|█████████ | 45/50 [01:48<00:17,  3.59s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [02:08<00:00,  2.57s/it]

Progress: [50/50]
Current Acc.: [18.00%]


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [22]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )
    prompt = "Instruction:\nSolve the following question by reasoning step-by-step. Show your work and conclude the answer after '####'.\n" #TODO: 프롬프트를 작성해주세요!

    for i in range(num_examples):
        #TODO: CoT example을 만들어주세요!
        idx = sampled_indices[i]
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"


    return prompt

In [ ]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

In [23]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_CoT_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_0.txt")

 10%|█         | 5/50 [00:05<00:53,  1.19s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:09<00:34,  1.18it/s]

Progress: [10/50]
Current Acc.: [60.00%]


 30%|███       | 15/50 [00:13<00:29,  1.18it/s]

...

Progress: [40/50]
Current Acc.: [55.00%]


 90%|█████████ | 45/50 [01:25<00:18,  3.64s/it]

Progress: [45/50]
Current Acc.: [55.56%]


100%|██████████| 50/50 [01:42<00:00,  2.05s/it]

Progress: [50/50]
Current Acc.: [58.00%]


In [24]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_CoT_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_3.txt")

 10%|█         | 5/50 [00:04<00:38,  1.16it/s]

Progress: [5/50]
Current Acc.: [60.00%]


 20%|██        | 10/50 [00:27<03:46,  5.65s/it]

Progress: [10/50]
Current Acc.: [50.00%]


 30%|███       | 15/50 [01:10<04:52,  8.35s/it]

...

Progress: [40/50]
Current Acc.: [60.00%]


 90%|█████████ | 45/50 [05:21<00:42,  8.45s/it]

Progress: [45/50]
Current Acc.: [62.22%]


100%|██████████| 50/50 [06:04<00:00,  7.30s/it]

Progress: [50/50]
Current Acc.: [62.00%]


In [25]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_CoT_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "CoT_prompting_5.txt")

 10%|█         | 5/50 [00:38<07:04,  9.43s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [01:38<07:54, 11.87s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [02:39<07:11, 12.33s/it]

...

Progress: [40/50]
Current Acc.: [65.00%]


 90%|█████████ | 45/50 [08:21<00:57, 11.48s/it]

Progress: [45/50]
Current Acc.: [66.67%]


100%|██████████| 50/50 [09:19<00:00, 11.20s/it]

Progress: [50/50]
Current Acc.: [68.00%]


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [44]:
### 자유롭게 수정해도 됩니다! 완전히 새로 함수를 만들어도 돼요.
def construct_my_prompt(num_examples: int = 3):
    # TODO: 구현해주세요!
    train_dataset = gsm8k_train

    prompt = (
        "You are a world-renowned mathematics coach known for your logical precision and clarity. One mistake means elimination.\n"
        "For each question, follow these steps carefully:\n"
        "1. Break down the problem into clear, logical steps.\n"
        "2. Perform all necessary calculations accurately.\n"
        "3. Before finalizing your answer, **double-check your reasoning for possible errors or oversights.**\n"
        "4. Conclude by stating ONLY the final numeric answer after the word 'Answer:' using the format '#### [number]'.\n"
        "IMPORTANT: Do not explain the final answer after '####'. The answer must be a single number.\n"
        "\n"
        "Use this format exactly:\n"
        "Step 1: ...\n"
        "Step 2: ...\n"
        "...\n"
        "Answer: #### [FINAL ANSWER]\n"
    )

    sampled_indices = random.sample(range(len(train_dataset)), num_examples)
    for i, idx in enumerate(sampled_indices):
        question = train_dataset[idx]['question']
        answer = train_dataset[idx]['answer'].strip()
        prompt += f"\n[Example {i+1}]\nQuestion:\n{question}\n{answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    return prompt

In [ ]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

In [45]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_my_prompt(0)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_0.txt")

 10%|█         | 5/50 [00:03<00:27,  1.65it/s]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:07<00:29,  1.33it/s]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:13<00:36,  1.04s/it]

...

Progress: [40/50]
Current Acc.: [72.50%]


 90%|█████████ | 45/50 [01:56<00:19,  3.90s/it]

Progress: [45/50]
Current Acc.: [71.11%]


100%|██████████| 50/50 [02:19<00:00,  2.79s/it]

Progress: [50/50]
Current Acc.: [70.00%]


In [51]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_my_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_3.txt")

 10%|█         | 5/50 [00:03<00:33,  1.36it/s]

Progress: [5/50]
Current Acc.: [100.00%]


 20%|██        | 10/50 [00:26<03:34,  5.36s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 24%|██▍       | 12/50 [00:34<02:47,  4.40s/it]

...

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1rcmwszf8kt32egqmjbh5e8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500180, Requested 672. Please try again in 2m27.3442s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1rcmwszf8kt32egqmjbh5e8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500179, Requested 654. Please try again in 2m24.074799999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


 96%|█████████▌| 48/50 [01:41<00:00,  6.29it/s]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1rcmwszf8kt32egqmjbh5e8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500178, Requested 603. Please try again in 2m15.101s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


100%|██████████| 50/50 [01:41<00:00,  2.04s/it]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1rcmwszf8kt32egqmjbh5e8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500177, Requested 599. Please try again in 2m14.207799999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1rcmwszf8kt32egqmjbh5e8` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500176, Requested 606. Please try again in 2m15.1614s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


In [47]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_my_prompt(5)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=50
)
save_final_result(results, accuracy, "My_prompting_5.txt")

 10%|█         | 5/50 [00:46<07:48, 10.41s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [01:45<07:48, 11.70s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [02:46<07:05, 12.15s/it]

...

Progress: [40/50]
Current Acc.: [72.50%]


 90%|█████████ | 45/50 [08:30<00:58, 11.77s/it]

Progress: [45/50]
Current Acc.: [73.33%]


100%|██████████| 50/50 [09:37<00:00, 11.55s/it]

Progress: [50/50]
Current Acc.: [74.00%]


### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!